In [1]:
from tensorflow.python.layers.convolutional import Conv2D
from tensorflow.python.layers.pooling import MaxPooling2D
from tensorflow.python.training.adadelta import AdadeltaOptimizer
from tensorflow.python.training.adagrad import AdagradOptimizer
from tensorflow.python.training.optimizer import Optimizer
from dopapy.core.services.instance.ai.tfmodel.gradient import GradientLearningTFModelServiceInstance
from dopapy.core.services.instance.ai.tfmodel.ilearning import ITFLearningModel
from dopapy.utils.common.guid import GUID
from tensorflow.contrib.eager.python.network import Network
from tensorflow.contrib.eager.python.tfe import enable_eager_execution
from tensorflow.python.layers.core import Dense, Dropout, Flatten
from tensorflow.python.training.adam import AdamOptimizer
from dopapy import *
from dopapy.trading import *
import dopapy.types as dd
import warnings
warnings.filterwarnings('ignore') #Thats just to ignore warnings that are irrelevant for this type of sample
enable_eager_execution()

class VisualCortexNetwork(Network):

    def __init__(self):
        super().__init__()
        #self._layer = self.track_layer(Dense(units=64))
        self.track_layer(Conv2D(filters=8, kernel_size=(3, 3)))
        self.track_layer(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        self.track_layer(Dropout(0.25))
        self.track_layer(Flatten())

    def call(self, inputs, **kwargs):
        result = inputs
        for layer in self.layers:
            result = layer(result)
        print("VisualCortex: {0}->{1}".format(inputs.shape,result.shape))
        return result

class VisualCortexModel(ITFLearningModel):

    def __init__(self):
        super().__init__()
        self._network = VisualCortexNetwork()
        self._optimizer = AdagradOptimizer(learning_rate=1e-2)

    def get_network(self) -> Network:
        return self._network

    def get_optimizer(self) -> Optimizer:
        return self._optimizer

class VisualCortexCreator(IServiceInstanceCreator):
    def create_instance(self, service_instance_id: GUID) -> GradientLearningTFModelServiceInstance:
        return GradientLearningTFModelServiceInstance(VisualCortexModel(), service_instance_id)


def get_visualcortex_session():
    print("preparing VisualCortex session...")
    in_advance = 100
    init_logging()
    input_descripotr = dd.TensorDescriptor(size=[-1,25,25,16], store_file_prefix="visualcortex")
    output_descriptor = dd.TensorDescriptor(size=[-1,3872])
    reward_desc = create_reward_descriptor(min_dopamine_price=2)
    reward_desc.add_payment(dd.RewardPaymentType.FORWARD, min_price=0, in_advance_quantity=1)
    reward_desc.add_payment(dd.RewardPaymentType.RETURN_GRADIENT, min_price=2,
                            in_advance_quantity=in_advance)
    reward_desc.add_payment(dd.RewardPaymentType.UPDATE, min_price=2,
                            in_advance_quantity=in_advance)
    service_type = dd.ServiceType.Learning.GRADIENT
    creator = VisualCortexCreator()
    service_desc = dd.ServiceDescriptor(input_descriptors=[input_descripotr],
                                                output_descriptors=[output_descriptor],
                                                service_role=dd.ServiceRole.PROVIDER,
                                                service_type=service_type)
    visualcortex_session = create_session('password123456', '/data/dopamine/keystore_dir_a/',
                                      '127.0.0.1',
                                      int(100E18), int(10E18), web_server_port=8501)
    visualcortex_services = create_quote_services(
                                              session=visualcortex_session,
                                              qty=1,
                                              side=SELL,
                                              service_descriptor=service_desc,
                                              reward_descriptor=reward_desc,
                                              service_instance_creator=creator)
    visualcortex_services.publish()
    print("Done")
    return visualcortex_session

In [2]:
vs = get_visualcortex_session()

preparing VisualCortex session...
Done
